### Practical 8

### Analyse sentiments based on subjects from  movie_reviews dataset in nltk

In [18]:
import nltk
import random
from nltk.corpus import movie_reviews

In [19]:
documents = [(list(movie_reviews.words(fileid)), category) for category in movie_reviews.categories() 
             for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)

In [20]:
print(documents[0]) # gives ([tokenized_words_of_review], label_pos_or_neg)

(['virtual', 'reality', 'is', 'a', 'topic', 'that', 'has', 'almost', 'defined', 'its', 'own', 'genre', '.', 'since', "'", 'dark', 'city', "'", '(', '1998', ')', ',', 'films', 'about', 'alternative', 'universes', 'and', 'parallel', 'worlds', 'have', 'been', 'coming', 'one', 'after', 'another', '.', 'the', 'most', 'popular', 'and', 'original', 'of', 'them', 'all', 'has', 'been', 'the', 'matrix', '.', 'all', 'the', 'films', 'released', 'after', 'it', ',', 'have', 'been', 'less', 'successful', '.', 'david', 'cronenberg', "'", 's', 'existenz', ',', 'released', 'only', 'a', 'couple', 'of', 'months', 'after', 'the', 'matrix', ',', 'was', 'accepted', 'and', 'even', 'appreciated', '(', 'by', 'some', ')', ',', 'but', 'the', 'hype', 'was', 'gone', '.', 'and', 'when', "'", 'the', 'thirteenth', 'floor', "'", 'aired', 'this', 'fall', ',', 'it', 'was', 'completely', 'overseen', 'and', 'ignored', '.', 'i', 'admit', 'that', 'this', 'issue', 'has', 'become', 'something', 'between', 'tiring', 'and', 'ann

In [21]:
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_features = list(all_words.keys())[:2000]
#print(all_words)
#print()
#print(word_features)

In [22]:
def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
        return features
feature_set = [(document_features(doc), cat) for (doc, cat) in documents]
train_set, test_set = feature_set[100:], feature_set[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [23]:
print('Accuracy level =',nltk.classify.accuracy(classifier, test_set))

Accuracy level = 0.6


In [24]:
classifier.show_most_informative_features(5) # class labels = pos and neg

Most Informative Features
          contains(plot) = True              neg : pos    =      1.4 : 1.0
          contains(plot) = False             pos : neg    =      1.3 : 1.0


### Analyse tweet's sentiment based on sentences from twitter_samples dataset in nltk

In [25]:
from nltk.corpus import twitter_samples, stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier
import re, string

In [26]:
# Cleaning the tweet data
def remove_noise(tweet_tokens, stop_words=()):
    cleaned_tokens = []
    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z] | [0-9] | [$-_@.&+#] | [!*\(\),] | (?:%[0-9a-fA-F][0-9a-fA-F]))+', '', token)
        token = re.sub("(@[A-Za-z0-9_]+)", '', token)
        
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        
        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)
        
        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [27]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

In [28]:
from nltk.corpus import twitter_samples
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')[0]
print(tweet_tokens[0])

#FollowFriday


In [33]:
if __name__ == "__main__":

    stop_words = stopwords.words('english')

    positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
    negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

    positive_cleaned_tokens_list = []
    negative_cleaned_tokens_list = []

    for tokens in positive_tweet_tokens:
        positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    for tokens in negative_tweet_tokens:
        negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    all_pos_words = get_all_words(positive_cleaned_tokens_list)

    freq_dist_pos = FreqDist(all_pos_words)
    print(freq_dist_pos.most_common(10))
    
    positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
    negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)
    
    positive_dataset = [(tweet_dict, "Positive") for tweet_dict in positive_tokens_for_model]

    negative_dataset = [(tweet_dict, "Negative") for tweet_dict in negative_tokens_for_model]

    dataset = positive_dataset + negative_dataset
    
    random.shuffle(dataset)

    train_data = dataset[:7000]
    test_data = dataset[7000:]

    classifier = NaiveBayesClassifier.train(train_data)

    print("Accuracy is:", classify.accuracy(classifier, test_data))

    print(classifier.show_most_informative_features(10))

    custom_tweet = "I ordered just once from TerribleCo, they were very nice."

    custom_tokens = remove_noise(word_tokenize(custom_tweet))

    print(custom_tweet, classifier.classify(dict([token, True] for token in custom_tokens)))

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]
Accuracy is: 0.9946666666666667
Most Informative Features
                      :) = True           Positi : Negati =   1007.7 : 1.0
                follower = True           Positi : Negati =     23.8 : 1.0
                     sad = True           Negati : Positi =     22.9 : 1.0
                     bam = True           Positi : Negati =     21.4 : 1.0
                  arrive = True           Positi : Negati =     21.0 : 1.0
                 welcome = True           Positi : Negati =     15.9 : 1.0
                     x15 = True           Negati : Positi =     14.1 : 1.0
                    blog = True           Positi : Negati =     13.2 : 1.0
                    glad = True           Positi : Negati =     12.8 : 1.0
                     via = True           Positi : Negati =     11.6 : 1.0
None
I ordered just once from TerribleCo, 

In [30]:
print(dataset)

[({'#followfriday': True, 'top': True, 'engage': True, 'member': True, 'community': True, 'week': True, ':)': True}, 'Positive'), ({'hey': True, 'james': True, 'odd': True, ':/': True, 'please': True, 'call': True, 'contact': True, 'centre': True, '02392441234': True, 'able': True, 'assist': True, ':)': True, 'many': True, 'thanks': True}, 'Positive'), ({'listen': True, 'last': True, 'night': True, ':)': True, 'bleed': True, 'amazing': True, 'track': True, 'scotland': True}, 'Positive'), ({'congrats': True, ':)': True}, 'Positive'), ({'yeaaaah': True, 'yippppy': True, 'accnt': True, 'verify': True, 'rqst': True, 'succeed': True, 'get': True, 'blue': True, 'tick': True, 'mark': True, 'fb': True, 'profile': True, ':)': True, '15': True, 'day': True}, 'Positive'), ({'one': True, 'irresistible': True, ':)': True, '#flipkartfashionfriday': True, 'http://t.co/ebz0l2venm': True}, 'Positive'), ({'like': True, 'keep': True, 'lovely': True, 'customer': True, 'wait': True, 'long': True, 'hope': T